Let's visualize the Mandebrot set. 

The Mandebrot set is the set of $c$ such that the function $f(z) = z^2 + c$ won't diverge when starting it with $z = 0$.

Let's be precies and define diverge means that the number goes out of the circle $|z| < R$ under $N$ iterations. 

**Is there an easier way to simplify this instead of just iterating all the numbers we are intersted in**? 
* Consder Dynamic Programming
* if $c_0$ diverges from previous iterations, then whenever we get close to that point, in another iterations of $k_0$, we mark this iterations as divergence as well. 
* This is bad, for 2 reasons: 
    1. It's a fractal. 
    2. How many iterations until divergence when we last iterated near the point $c_0$? How many iteration remains for $k_0$ when it gets into the region of $c_0$? If iterations starts at $c_0, c_1, c_2...$, and it diverges, we might need to record the number of iterations until it diverges for all those points. It's just complicated to code it. And we need a very fine discretization of the complex plane to make this works. 
    



In [1]:
using Colors, Images, ImageView

function Manderbrot_Iterate(c0::Complex{Float64}; maxItr::Int64 = 1024, radius::Number = 2)
    zk = c0
    counter = 0
    while(abs(zk) < radius &&  counter < maxItr)
        zk = zk^2 + c0
        counter += 1
    end
    return counter # maxtr means in the set, else it's... not quiet in the set. 
end

"""
Given a region in the complex plane, discritized into a grids of a certain height, and width. 
We are interested in iterating all of them and make a coordinate iteration matrix 
out of it. 
"""
function Offset_Gridpoints(topLeft:: Tuple, bottomRight:: Tuple; 
                                width::Int64 = 1080, height::Int64 = 1920)
    
    dx = (bottomRight[1] - topLeft[1])/width
    dy = (topLeft[2] - bottomRight[2])/height
    Ygrid = topLeft[2] - dy/2: -dy: bottomRight[2] + dy/2
    Xgrid = topLeft[1] + dx/2: dx: bottomRight[1] - dx/2
    Xgrid = collect(Xgrid)
    Ygrid = collect(Ygrid)
    
    Ygrid = reshape(Ygrid, length(Ygrid), 1)
    Xgrid = reshape(Xgrid, 1, length(Xgrid))

    Ygrid = Ygrid .+ zeros(width, height)
    Xgrid = Xgrid .+ zeros(width, height)
    Plane = Xgrid + Ygrid.*im
    return Plane
end

Offset_Gridpoints

In [52]:
ComplexPlane = Offset_Gridpoints((-2.5, 2), (1.5, -2); width=4000, height=4000)
ItrMatrix = Manderbrot_Iterate.(ComplexPlane; radius=4)
ItrMatrix = abs.(log.(ItrMatrix))
ItrMatrix = ItrMatrix./maximum(ItrMatrix)

img = Gray.(ItrMatrix)
save("Manderbrot.png", img)

In [5]:
ComplexPlane = Offset_Gridpoints((-2.5, 2), (1.5, -2); width=5000, height=5000)
ItrMatrix = Array{Int64}(undef, size(ComplexPlane))

Threads.@threads for II = eachindex(ItrMatrix)
    ItrMatrix[II] = Manderbrot_Iterate(ComplexPlane[II], radius=4)
end

ItrMatrix = abs.(log.(ItrMatrix))
ItrMatrix = ItrMatrix./maximum(ItrMatrix)

img = Gray.(ItrMatrix)
save("Manderbrot.png", img)